statsmodels使用endog和exog作为数据的名称，即用于估计问题的观察变量。其他经常在不同的统计软件包或教科书中使用的名称.

它的使用通常是特定于领域和模型的;然而，我们几乎只选择了endog和exog。让这两项分开的一个便于记忆的提示是外生的名称中有一个“x”，就像x变量一样。

x和y是一个字母名称，有时用于临时变量，本身没有信息。为了避免一个字母的名字，我们决定使用描述性的名字，并确定endog和exog。由于这一点受到了批评，今后可能会有所改变。

## 背景

这些术语的一些非正式定义是

内源性:由系统内部因素引起的

外生的:由系统外部因素引起的

内生变量指的是在经济/计量经济模型中由该模型解释或预测的变量。http://stats.oecd.org/glossary/detail.asp?ID=794

外生变量指的是出现在经济/计量模型中但不被该模型解释的变量(即，它们被视为模型给出的变量)。http://stats.oecd.org/glossary/detail.asp?ID=890

在计量经济学和统计学中，术语的定义更为正式，并且根据模型使用不同的外生性定义(弱、强、严格)。statsmodels中作为变量名的使用不能总是以正式的意义来解释，但是尝试遵循同样的原则。

最简单的形式是，模型以线性或非线性的形式将观察到的变量y与另一组变量x联系起来

然而，要建立一个统计模型，我们需要对解释变量x和噪声的属性进行额外的假设。许多基本模型的一个标准假设是x与噪声无关。在更一般的定义,x外生意味着我们不必考虑x的解释变量是如何生成的,不管是设计还是由一些随机吸引潜在的分布,当我们想要估计的影响或影响x y,或测试假说对这个效果。

换句话说，y对于我们的模型是内生的，而x对于我们的估计模型是外生的。

例如，假设您进行了一个实验，在第二个阶段，一些主题不再可用。退出与你在实验中得出的结论相关吗?换句话说，对于我们的问题，我们是否可以将退出决策视为外生因素。

这些模型的基本统计假设是什么，这取决于用户(或者通过查阅教科书来找出)。例如，如果误差或噪声项随时间独立分布(或不相关)，OLS中的exog可能有滞后因变量。但是，如果误差项是自相关的，那么OLS就没有很好的统计特性(is inconsistent)，正确的模型就是ARMAX。statsmodels具有用于回归诊断的功能，以测试某些假设是否合理。

## 导入路径和结构

我们提供了两种从statsmodel导入函数和类的方法:

API导入用于交互使用

程序的直接导入

API Import for interactive use：

API导入交互使用：

In [2]:
import statsmodels.api as sm

import statsmodels.api将加载状态模型的大部分公共部分。这使得大多数函数和类可以在一到两层中方便地使用，而不会使“sm”名称空间过于拥挤。

程序直接导入

In [5]:
from statsmodels.tools.tools import add_constant

## 使用R形公式拟合模型

自从0.5.0版本以来，statsmodels允许用户使用r风格的公式来适应统计模型。在内部，statsmodels使用patsy包将公式和数据转换为用于模型拟合的矩阵。公式框架非常强大;本教程只触及表面。关于公式语言的详细描述可以在patsy文档中找到:

注意，我们调用了statsmodels.formula.api。除了通常的statsmodel .api之外的api。事实上,statsmodel .api这里只使用api来加载数据集。这个formula.api承载了api中发现的许多相同的函数(例如OLS、GLM)，但是它也为大多数这些模型保存了小写的对应函数。

**一般情况下，小写模型接受公式和df参数，而大写模型则接受endog和exog设计矩阵。formula接受一个字符串，该字符串用patsy公式描述模型。df取一个pandas dataframe**

dir(smf)将打印可用模型的列表。

公式兼容模型有以下通用调用签名:(formula, data, subset=None, *args, **kwargs)

查看上面打印的摘要，请注意，patsy将Region的元素确定为文本字符串，因此它将Region视为一个分类变量。patsy的默认设置还包括一个截距（intercept），所以我们自动删除了一个区域类别。

如果Region是一个整型变量，我们想明确地将其视为分类变量，我们可以使用C()操作符来实现:

我们已经看到，“~”分隔了模型的左边和右边，“+”向设计矩阵添加了新的列。

移除变量

符号"-"可以用来删除列/变量。例如，我们可以通过以下方法从模型中删除截距:

In [16]:
import pandas as pd
import statsmodels.formula.api as smf
df = pd.read_csv('Rdatasets-master/csv/HistData/Guerry.csv',
                index_col=0)
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()

In [17]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) -1 ', data=df).fit()

In [18]:
res.params

C(Region)[C]    38.651655
C(Region)[E]    23.223870
C(Region)[N]    28.634694
C(Region)[S]    34.103399
C(Region)[W]    28.560379
Literacy        -0.185819
Wealth           0.451475
dtype: float64

乘法交互项：

In [19]:
res1 = smf.ols(formula='Lottery ~ Literacy : Wealth - 1', data=df).fit()
res2 = smf.ols(formula='Lottery ~ Literacy * Wealth - 1', data=df).fit()

In [20]:
res1.params

Literacy:Wealth    0.018176
dtype: float64

In [21]:
res2.params

Literacy           0.427386
Wealth             1.080987
Literacy:Wealth   -0.013609
dtype: float64

你可以对模型中的变量应用向量化函数:

In [23]:
import numpy as np
res = smf.ols(formula='Lottery ~ np.log(Literacy)', data=df).fit()

In [24]:
res.params

Intercept           115.609119
np.log(Literacy)    -20.393959
dtype: float64

In [25]:
def log_plus_1(x):
   return np.log(x) + 1.0

In [26]:
res = smf.ols(formula='Lottery ~ log_plus_1(Literacy)', data=df).fit()

In [27]:
res.params

Intercept               136.003079
log_plus_1(Literacy)    -20.393959
dtype: float64

命名空间

使用公式和(还)不支持它们的模型

即使给定的statsmodels函数不支持公式，您仍然可以使用patsy的公式语言来生成设计矩阵。这些矩阵可以作为endog和exog参数被输入到拟合函数中。

生成numpy数组:

In [29]:
import patsy
f = 'Lottery ~ Literacy * Wealth'
y, X = patsy.dmatrices(f, df, return_type='matrix')

In [32]:
X[:5]

array([[1.000e+00, 3.700e+01, 7.300e+01, 2.701e+03],
       [1.000e+00, 5.100e+01, 2.200e+01, 1.122e+03],
       [1.000e+00, 1.300e+01, 6.100e+01, 7.930e+02],
       [1.000e+00, 4.600e+01, 7.600e+01, 3.496e+03],
       [1.000e+00, 6.900e+01, 8.300e+01, 5.727e+03]])

生成dataframe:

In [33]:
y, X = patsy.dmatrices(f, df, return_type='dataframe')
X[:5]

,Intercept,Literacy,Wealth,Literacy:Wealth
1,1.0,37.0,73.0,2701.0
2,1.0,51.0,22.0,1122.0
3,1.0,13.0,61.0,793.0
4,1.0,46.0,76.0,3496.0
5,1.0,69.0,83.0,5727.0


## Pitfalls

使用状态模型时可能出现的问题:

- 这些可能是与数据相关或统计问题、软件设计、模型的“非标准”使用或边缘情况的结果。

- statsmodel提供了几个用于诊断检查的警告和帮助函数(有关线性回归中错误规范检查的示例，请参阅这篇博客文章)。当然，其覆盖范围并不全面，但随着时间的推移，将添加更多的警告和诊断功能。

- 虽然所有统计包的潜在统计问题都是相同的，但是软件实现在处理极端或极端情况的方式上是不同的。请报告模型可能不起作用的角落案例，以便我们适当处理。

- 重复调用以适应不同的参数

 - **结果实例经常需要访问来自相应模型实例的属性。用不同的参数多次拟合一个模型可能会改变模型的属性。这意味着在模型重新匹配之后，result实例可能不再指向正确的模型属性。**

 - **因此，当我们希望使用不同的fit函数参数来拟合一个模型时，最佳实践是创建单独的模型实例。**

 - 例如，这样做没有问题，因为我们没有保留results实例供进一步使用

例如，这样做没有问题，因为我们没有保留results实例供进一步使用

from statsmodels.tsa.ar_model import AR
endog = [1,2]
mod = AR(endog)
aic = []
for lag in range(1,11):
    res = mod.fit(maxlag=lag)
    aic.append(res.aic)

但是，当我们希望保留两个不同的估计结果时，建议创建两个单独的模型实例。

mod1 = RLM(endog, exog)
res1 = mod1.fit(scale_est='mad')
mod2 = RLM(endog, exog)
res2 = mod2.fit(scale_est=sm.robust.scale.HuberScale())